## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
split_names = list(dataset.keys())

In [2]:
import numpy as np
from termcolor import cprint
split_names = list(dataset.keys())
for split_name in split_names:
    cprint(f"Split name : {split_name}", attrs=['bold'])
    cprint(dataset[split_name])
    pos_revs = np.array(dataset[split_name]['label']) == 1
    neg_revs = ~pos_revs
    cprint(f"Number of positive reviews: {pos_revs.sum()}", attrs=['bold'], color='green')
    cprint(f"Number of negative reviews: {neg_revs.sum()}", attrs=['bold'], color='red')
    print()

trainset, valset, testset = (dataset[split_name] for split_name in split_names)


Split name : train
Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})
Number of positive reviews: 4265
Number of negative reviews: 4265

Split name : validation
Dataset({
    features: ['text', 'label'],
    num_rows: 1066
})
Number of positive reviews: 533
Number of negative reviews: 533

Split name : test
Dataset({
    features: ['text', 'label'],
    num_rows: 1066
})
Number of positive reviews: 533
Number of negative reviews: 533



#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")                                        
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Get a few sample texts from the training dataset
samples = np.random.choice(trainset, size=5, replace=False)
texts = [sample['text'] for sample in samples]
labels = [sample['label'] for sample in samples]
cprint("Sample texts:", attrs=['bold'])
for text, label in zip(texts, labels):
    color = 'green' if label == 1 else 'red'
    cprint(f"- {text}", attrs=['bold'], color=color)

cprint("-" * 30, attrs=['bold'])

tokenized_inputs = tokenizer(texts,truncation=True,padding=True,return_tensors='pt')

print(tokenized_inputs['input_ids'].shape)
print(tokenized_inputs['attention_mask'].shape)

outputs = model(**tokenized_inputs)

print(outputs.last_hidden_state.shape)

Sample texts:
- a certain sexiness underlines even the dullest tangents .
- an enthralling , playful film that constantly frustrates our desire to know the 'truth' about this man , while deconstructing the very format of the biography in a manner that derrida would doubtless give his blessing to .
- one-sided documentary offers simplistic explanations to a very complex situation . . . . stylistically , the movie is a disaster .
- a savage john waters-like humor that dances on the edge of tastelessness without ever quite falling over .
- in all fairness , i must report that the children of varying ages in my audience never coughed , fidgeted or romped up and down the aisles for bathroom breaks .
------------------------------
torch.Size([5, 53])
torch.Size([5, 53])
torch.Size([5, 53, 768])
torch.Size([5, 53, 768])


#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [4]:
from transformers import pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import torch

# how to figure out which token is the [CLS] token
for i in range(len(tokenized_inputs["input_ids"])):
    id_to_token = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i])
    arg_to_token = np.argmax(id_to_token == "[CLS]")
    print("Token : ",id_to_token[arg_to_token], "at position : ", arg_to_token)
# it's at position 0 in the sequence dimension



Token :  [CLS] at position :  0
Token :  [CLS] at position :  0
Token :  [CLS] at position :  0
Token :  [CLS] at position :  0
Token :  [CLS] at position :  0


In [5]:
device = 0 if torch.cuda.is_available() else -1
model.to(device)


def extract_features_batched(dataset, tokenizer, model, device, batch_size,output_path):
    features_list = []
    for i in range(0, len(dataset["text"]), batch_size):
        batch_texts = dataset["text"][i : i + batch_size]
        tokenized_inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
        tokenized_inputs = {k: v.to(device) for k, v in tokenized_inputs.items()}
        with torch.no_grad():
            outputs = model(**tokenized_inputs)
        class_token = outputs.last_hidden_state[:,0,:]
        features_list.append(class_token.cpu()) 
    
    all_features = torch.cat(features_list, dim=0)
    
    try:
        torch.save(all_features, output_path)
        print(f"Saved features to {output_path}")
    except Exception as e:
        print(f"Error saving features to {output_path}: {e}")

    return all_features


val_path = "val_features.pth"
val_features = extract_features_batched(valset, tokenizer, model, device, batch_size=64,output_path=val_path)
train_path = "train_features.pth"
train_features = extract_features_batched(trainset, tokenizer, model, device, batch_size=64,output_path=train_path)
test_path = "test_features.pth"
test_features = extract_features_batched(testset, tokenizer, model, device, batch_size=64,output_path=test_path)       



Saved features to val_features.pth
Saved features to train_features.pth
Saved features to train_features.pth
Saved features to test_features.pth
Saved features to test_features.pth


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def train_and_evaluate_svm(train_features, train_labels, val_features, val_labels):
    
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto', 0.1, 0.01]
    }
    
    grid_search = GridSearchCV(
        SVC(),
        param_grid,
        cv=5,
        scoring='accuracy',
        verbose=1,
        n_jobs=-1
    )
    
    grid_search.fit(train_features, train_labels)
    
    svm = grid_search.best_estimator_
    
    cprint(f"Best parameters: {grid_search.best_params_}", attrs=['bold'], color='green')
    
    val_predictions = svm.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)
    
    cprint(f"Validation Accuracy: {accuracy:.4f}", attrs=['bold'], color='blue')
    
    return svm

train_features = torch.load(train_path)
train_labels = np.array(trainset['label'])
val_features = torch.load(val_path)
val_labels = np.array(valset['label'])

svm_model = train_and_evaluate_svm(train_features, train_labels, val_features, val_labels)


In [ ]:
# # Logging to wandb  
# import wandb
# wandb.login(key="##############################")  
# wandb.init(project="lab-3", name="svm-rotten-tomatoes")

# wandb.log({
#     "best_params": svm_model.get_params(),
#     "validation_accuracy": accuracy_score(val_labels, svm_model.predict(val_features))
# })
# wandb.finish()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Currently logged in as: francesco-correnti (francesco-correnti-unifi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: francesco-correnti (francesco-correnti-unifi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [8]:

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length")

tokenized_train = trainset.map(tokenize_function, batched=True)
tokenized_val = valset.map(tokenize_function, batched=True)
tokenized_test = testset.map(tokenize_function, batched=True)

print(tokenized_train.features)
print(f"First example features: {list(tokenized_train[0].keys())}")

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
First example features: ['text', 'label', 'input_ids', 'attention_mask']

First example features: ['text', 'label', 'input_ids', 'attention_mask']


#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [9]:
from transformers import AutoModelForSequenceClassification

model_clsfy = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=2
)

model_clsfy.to(device)

print(model_clsfy)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [ ]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import uuid

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted'),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted')
    }

run_id = str(uuid.uuid4())[:8]
wandb.init(project="lab-3", name=f"distbert-clsfy-{run_id}")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    warmup_steps=100,
    learning_rate=2e-5, 
    logging_dir='./logs',
    logging_steps=50,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    max_grad_norm=1.0,
    dataloader_drop_last=False,
    gradient_accumulation_steps=1,
    seed=42,  
)

from transformers.trainer_callback import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=50,
    early_stopping_threshold=0.001
)

trainer = Trainer(
    model=model_clsfy,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping], 
)

trainer.train()
results = trainer.evaluate()

wandb.log(results)
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.690800,0.679809,0.564728,0.465130,0.753673,0.564728
100,0.610300,0.498544,0.770169,0.764527,0.798804,0.770169
150,0.443300,0.457953,0.796435,0.796426,0.796486,0.796435
200,0.408800,0.423343,0.808630,0.807894,0.813436,0.808630
250,0.399200,0.392586,0.825516,0.824716,0.831566,0.825516
300,0.423000,0.367932,0.836773,0.836771,0.836792,0.836773
350,0.376100,0.365124,0.843340,0.843290,0.843776,0.843340
400,0.392800,0.345046,0.848968,0.848871,0.849866,0.848968
450,0.351800,0.346111,0.851782,0.851778,0.851827,0.851782
500,0.365800,0.356504,0.849906,0.849803,0.850874,0.849906


epoch,▁
eval/accuracy,▁▆▆▇▇▇████████████████
eval/f1,▁▆▇▇▇█████████████████
eval/loss,█▄▃▃▂▁▁▁▁▁▁▂▂▂▂▂▂▁▁▁▁▁
eval/precision,▁▄▄▅▆▆▇▇▇▇▇█▇▇▇███████
eval/recall,▁▆▆▇▇▇████████████████
eval/runtime,▄▁▁▂▁▁▂▃▂▂▃▃▃▆█▇▆▇▇▇▇▇
eval/samples_per_second,▅██▇██▇▆▇▇▆▆▆▃▁▂▃▂▂▂▂▂
eval/steps_per_second,▅██▇██▇▆▇▇▆▆▆▃▁▂▃▂▂▂▂▂
eval_accuracy,▁
eval_f1,▁


-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [11]:
# Your code here.

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [12]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [ ]:
# code in Lab3-Transformers-Exercise-3.3.ipynb

: 